Tokenization can have a significant impact on the quality of models. In this task, you will need to reduce the tokenizer to reduce the size of the model while maintaining quality. You will solve the problem of sentiment analysis. In the first part of the notebook, we implement a baseline, which measures the quality of the original model. In the second part, you need to reduce the tokenizer by removing at least 50% of tokens from it, reinitialize the embedding layer of BERT model and retrain the model. The classification quality should decrease by no more than 2% in terms of the F-measure, compared to the original model.

### Baseline

In [ ]:
!pip install transformers datasets evaluate accelerate tokenizers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    BertTokenizerFast
)
from datasets import load_dataset
import evaluate
import os
from tokenizers import BertWordPieceTokenizer

In [ ]:
# 1. Загрузка данных
dataset = load_dataset("imdb")
train_data = dataset["train"].shuffle(seed=42).select(range(5000))
test_data = dataset["test"].shuffle(seed=42).select(range(1000))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# 2. Исходный токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

train_dataset = train_data.map(tokenize_function, batched=True)
test_dataset = test_data.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# 3. Обучение исходной модели
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)

model.cuda()

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to=['tensorboard'],
    bf16=True,
)

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.393300,0.357452,0.871000
2,0.243900,0.419015,0.886000
3,0.136900,0.506714,0.889000


TrainOutput(global_step=1875, training_loss=0.22290226236979166, metrics={'train_runtime': 1051.8865, 'train_samples_per_second': 14.26, 'train_steps_per_second': 1.783, 'total_flos': 1973332915200000.0, 'train_loss': 0.22290226236979166, 'epoch': 3.0})

In [ ]:
# 4. Оценка исходной модели
original_accuracy = trainer.evaluate()["eval_accuracy"]
print(f"Исходная точность: {original_accuracy:.4f}")

Исходная точность: 0.8890


### Task 1

Remove at least 50% of tokens from original tokenizer. Reinitialize new tokenizer. Give to new tokenizer 'new_tokenizer' name.

In [ ]:
from collections import Counter
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizerFast

# 5. Подсчет частоты токенов
def get_token_frequencies(dataset, tokenizer):
    counter = Counter()
    for example in dataset["text"]:
        tokens = tokenizer.tokenize(example)
        counter.update(tokens)
    return counter

token_freqs = get_token_frequencies(train_data, tokenizer)

# 6. Оставляем 50% самых популярных токенов
vocab_size = len(tokenizer.vocab)
num_to_keep = round(vocab_size / 2.001)
most_common_tokens = {token for token, _ in token_freqs.most_common(num_to_keep)}

# Фильтруем данные, оставляя только предложения с нужными токенами
filtered_texts = [
    example for example in train_data["text"]
    if any(token in most_common_tokens for token in tokenizer.tokenize(example))
]

# 7. Создаем новый токенизатор
bert_wordpiece_tokenizer = BertWordPieceTokenizer()
bert_wordpiece_tokenizer.train_from_iterator(filtered_texts, vocab_size=num_to_keep + len(tokenizer.all_special_tokens))

# Создаем объект BertTokenizerFast
new_tokenizer = BertTokenizerFast(tokenizer_object=bert_wordpiece_tokenizer)

# Настраиваем усечение и дополнение
new_tokenizer.truncation = True
new_tokenizer.padding = 'max_length'
new_tokenizer.model_max_length = 256

# Выводим размеры словаря
print(f"\nРазмеры словарей:")
print(f"Исходный: {len(tokenizer.vocab)}")
print(f"Новый: {len(new_tokenizer.vocab)}")
print(f"Удалено токенов: {len(tokenizer.vocab) - len(new_tokenizer.vocab)}")
print(f"Удалено токенов в %: {(len(tokenizer.vocab) - len(new_tokenizer.vocab)) * 100 / len(tokenizer.vocab):.2f}%")



Размеры словарей:
Исходный: 30522
Новый: 15258
Удалено токенов: 15264
Удалено токенов в %: 50.01%


### Task 2:

Initialize new Embedding layer of BERT model according to new tokenizer.
Retrain model on classification task. The classification quality should decrease by no more than 2% in terms of the F-measure, compared to the original model.

In [ ]:
#TODO Your code is here


# 7. Переинициализация модели с новым словарем
new_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    ignore_mismatched_sizes=True  # Важно для изменения размера эмбеддингов!
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tqdm import tqdm

In [ ]:
# 8. Обучение с новым токенизатором
def new_tokenize_function(examples):
    tokens = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

    # Получаем новые id токенов
    new_vocab_set = set(new_tokenizer.get_vocab().values())

    # Фильтрация с использованием NumPy
    filtered_ids = [list(np.array(ids)[np.isin(ids, list(new_vocab_set))]) for ids in tokens["input_ids"]]

    # Дополняем до max_length
    max_len = 256
    tokens["input_ids"] = [ids + [new_tokenizer.pad_token_id] * (max_len - len(ids)) for ids in filtered_ids]

    return tokens




new_train_dataset = train_data.map(new_tokenize_function, batched=True)
new_test_dataset = test_data.map(new_tokenize_function, batched=True)

new_trainer = Trainer(
    model=new_model,
    args=training_args,
    train_dataset=new_train_dataset,
    eval_dataset=new_test_dataset,
    compute_metrics=compute_metrics,
)

new_trainer.train()
new_accuracy = new_trainer.evaluate()["eval_accuracy"]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,0.468200,0.396843,0.841000
2,0.289300,0.477887,0.872000
3,0.189200,0.543072,0.881000


In [ ]:
# 9. Результаты
print("\nРезультаты сравнения:")
print(f"Исходная точность: {original_accuracy:.4f}")
print(f"Точность с новым словарем: {new_accuracy:.4f}")
print(f"Разница: {original_accuracy - new_accuracy:.4f}")
print(f"Экономия памяти: {(len(tokenizer.vocab) - len(new_tokenizer.vocab))/len(tokenizer.vocab):.2%}")


Результаты сравнения:
Исходная точность: 0.8890
Точность с новым словарем: 0.8810
Разница: 0.0080
Экономия памяти: 50.01%
